# Implementación de un ChatBot sin memoria

En esta práctica se implementa un ChatBot sin memoria, haciendo uso de Gradio para la visualización de una interfaz gráfica, y de la API de OpenAI para responder las diferentes preguntas o cuestiones del usuario. 

Sin embargo, en este caso, no se incorpora memoria en el chatbot, haciendo llamadas aisladas al SDK de OpenAI. 

# Instalación de librerías

In [ ]:
!pip install openai==0.28
!pip install python-dotenv
!pip install gradio

# Importación de librerías

In [9]:
import gradio as gr
import openai
from dotenv import load_dotenv
import os

load_dotenv()

# Obtener las variables de entorno con los endpoints y claves.
openai.api_type = os.getenv("AZURE_OPENAI_API_TYPE")
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# Creación del chatbot sin memoria

In [ ]:

# Función para manejar las solicitudes del chatbot sin memoria
def chatbot_no_memory(message):
    # El mensaje se envía sin incluir historial previo
    messages = [{"role": "system", "content": "Eres un asistente útil y amigable."}]

    # Añadir el nuevo mensaje del usuario
    messages.append({"role": "user", "content": message})

    # Solicitud a OpenAI para generar la respuesta
    response = openai.ChatCompletion.create(
        engine="gpt-4-turbo",  # Nombre del deployment
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )

    # Obtener la respuesta del asistente
    bot_response = response['choices'][0]['message']['content']

    # Devolver solo la respuesta del bot (sin historial)
    return bot_response

# Definir la interfaz de Gradio
def create_chat_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Chatbot sin Memoria usando OpenAI")

        # Definir los componentes del chat
        chatbot = gr.Chatbot(label="Chat con OpenAI")
        msg = gr.Textbox(label="Tu mensaje")
        clear = gr.Button("Limpiar Chat")

        # Estado del historial de chat
        state = gr.State([])  # Estado para almacenar el historial del chat

        # Acción para enviar mensaje
        def send_message(user_message, chat_history):
            # Obtener la respuesta del chatbot sin utilizar historial
            bot_message = chatbot_no_memory(user_message)

            # Actualizar el historial de chat visible, pero sin afectar la memoria del bot
            chat_history.append((user_message, bot_message))

            # Devolver el mensaje del usuario y la respuesta acumulada en el historial
            return "", chat_history, chat_history

        # Conectar los componentes
        msg.submit(send_message, [msg, state], [msg, state, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)  # Limpiar el chat

    return demo


# Ejecución del chatbot

In [ ]:

# Ejecutar la interfaz
if __name__ == "__main__":
    demo = create_chat_interface()
    demo.launch()